In [ ]:
!pip install pandas transformers sentencepiece catboost nltk pymorphy2

In [30]:
from tqdm.notebook import tqdm
import umap
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
stopwords = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings("ignore")
import os
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
data = pd.read_csv("data/train.csv")[:50]
data

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
5,АО ПРО ТКО,Мусор/Свалки/ТКО,Добрый день.подскажите тариф повысился на обр...,Плата за вывоз ТКО
6,Министерство образования,Образование,"Здравствуйте, уточните в какую погоду дети на...",Образовательный процесс
7,Министерство здравоохранения,Здравоохранение/Медицина,"Пермский р н, 79519290579. Диспансеризация",Диспансеризация
8,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, скажите пожалуйста сколько врем...",★ Просьбы о лечении
9,ИГЖН ПК,Дороги,Хотел узнать в Чайковской управляйке по телеф...,★ Нарушение правил очистки дорог от снега и на...


# ОЧИСТКА ДЛЯ БЕРТА

In [51]:
def text_cleaner_for_bert(input_text: str) -> str:
    """ Функция, которая выполняет очистку текста от лишних символов """
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_url = re.sub(r'([a-z]{2}\d+[a-z]{2})', ' ', rem_url)
    rem_url = re.sub(r'!+', '.', rem_url)
    text = re.sub(
        r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+]+", " ", rem_url)
    text = re.sub("!", ".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]", '', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U0001F1F2-\U0001F1F4"  # Macau flag
                               u"\U0001F1E6-\U0001F1FF"  # flags
                               u"\U0001F600-\U0001F64F"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U0001F1F2"
                               u"\U0001F1F4"
                               u"\U0001F620"
                               u"\u200d"
                               u"\u2640-\u2642"
                               "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    
    return text


def bert_dataset_cleaner(data: pd.DataFrame) -> pd.DataFrame:
    data_frame = data.copy()
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = text_cleaner_for_bert(data_frame.loc[index, "Текст инцидента"]) 
    return data_frame
cleaned_dataset = bert_dataset_cleaner(data)

0it [00:00, ?it/s]

In [52]:
data

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
5,АО ПРО ТКО,Мусор/Свалки/ТКО,Добрый день.подскажите тариф повысился на обр...,Плата за вывоз ТКО
6,Министерство образования,Образование,"Здравствуйте, уточните в какую погоду дети на...",Образовательный процесс
7,Министерство здравоохранения,Здравоохранение/Медицина,"Пермский р н, 79519290579. Диспансеризация",Диспансеризация
8,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте, скажите пожалуйста сколько врем...",★ Просьбы о лечении
9,ИГЖН ПК,Дороги,Хотел узнать в Чайковской управляйке по телеф...,★ Нарушение правил очистки дорог от снега и на...


# TFIDF UMAP

In [45]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    #print(words)
    res = list()
    for word in words:
        if (word not in stopwords and len(word) > 1):
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    text = " ".join(res)
    return text

def text_cleaner_for_tfidf(input_text:str) -> str:
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_url= re.sub(r'([a-z]{2}\d+[a-z]{2})',' ',rem_url)
    rem_url = re.sub(r'!+','.', rem_url)
    rem_num = re.sub('[0-9]+', '', rem_url)
    text = re.sub(r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+.,!]+", " ", rem_num)
    text = re.sub("!",".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]",'', text)
    emoji_pattern=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    text = text.lower()
    return text


def dataset_preprocessor_vectorizers(data: pd.DataFrame) -> pd.DataFrame:
    data_frame = data.copy()

    indexes_to_drop = []
    # Regex для групп
    for index, row in tqdm(data_frame.iterrows()):
        # Очистка текста
        data_frame.loc[index, "Текст инцидента"] = text_cleaner_for_tfidf(data_frame.loc[index, "Текст инцидента"])
    # Лемматизация и удаление стоп слов
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = lemmatize(row["Текст инцидента"])
    return data_frame

cleaned_dataset_tfidf = dataset_preprocessor_vectorizers(data)


# TF IDF


class Model_tfidf_umap:
    def __init__(self, max_features=None, presaved=False, use_umap=True, vectorizer_umap_size=3):
        self.max_features = max_features
        self.vectorizer = TfidfVectorizer(max_features=self.max_features)
        self.vectorizer_name = "tfidf"
        self.vectorizer_umap_size = vectorizer_umap_size
        self.presaved = presaved
        self.use_umap = use_umap
        os.makedirs("./vectorizers/", exist_ok=True)
        os.makedirs("./umap/", exist_ok=True)
    
    def create_tfidf(self, data):
        if self.presaved:
            try:
                self.vectorizer = joblib.load(f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pkl')
                print(f"\n[LOG] Vectorizer loaded successfully")
                vocab = self.vectorizer.vocabulary_
                tf = self.vectorizer.transform(data)
                data_frame = pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys()))
                del tf
                return data_frame
            except Exception as e:
                error_name = type(e).name
                print(f"Caught an error: {error_name}")
                print(f"EXCEPTION: {e}")
                print(sys.exc_info()[0])
                traceback.print_exc(file=sys.stdout)
        else:
            print(f"\n[LOG] Vectorizer for params {self.vectorizer_name} and {self.max_features} is not found\n ")
            print(f"\n[LOG] Building new vectorizer")
            tf = self.vectorizer.fit_transform(data)
            vocab = self.vectorizer.vocabulary_
            joblib.dump(self.vectorizer, f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pkl')
            data_frame = pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys()))
            del tf

            return data_frame
    
    def create_umap(self,  data):
        if self.presaved:
            try:
                ump = joblib.load(f'umap/umap{self.vectorizer_umap_size}.sav')
                print(f"\n[LOG] UMAP loaded successfully")
                column_names = [f"umap_{i}" for i in range(self.vectorizer_umap_size)]
                data = ump.transform(data.values)
                
                return pd.DataFrame(data, columns=column_names)
            except Exception as e:
                error_name = type(e).name
                print(f"Caught an error: {error_name}")
                print(f"EXCEPTION: {e}")
                print(sys.exc_info()[0])
                traceback.print_exc(file=sys.stdout)
        else:
            print(f"\n[LOG] UMAP for param {self.vectorizer_umap_size}  is not found\n ")
            print(f"\n[LOG] Building new UMAP")
            column_names = [f"umap_{i}" for i in range(self.vectorizer_umap_size)]
            ump = umap.UMAP(n_components=self.vectorizer_umap_size, random_state=42)
            data = ump.fit_transform(data.values)
            joblib.dump(ump, f'umap/umap{self.vectorizer_umap_size}.sav')
            
            return pd.DataFrame(data, columns=column_names)
            
    def create_dataset(self, data):
        data_ = data.copy()
        
        data_tf_idf = self.create_tfidf(data_["Текст инцидента"].to_list())
        
        if self.use_umap:
            print("sas")
            data_umap = self.create_umap(data_tf_idf)
            data_ = pd.concat([data_, data_umap], axis=1)
            return data_
        else:
            data_ = pd.concat([data_, data_tf_idf], axis=1)

        return data_
    
mtf = Model_tfidf_umap(max_features=10, presaved=False, use_umap=False, vectorizer_umap_size=3)
res = mtf.create_dataset(cleaned_dataset_tfidf)

0it [00:00, ?it/s]

0it [00:00, ?it/s]


[LOG] Vectorizer for params tfidf and 10 is not found
 

[LOG] Building new vectorizer


In [46]:
res

,Исполнитель,Группа тем,Текст инцидента,Тема,день,добрый,документ,дом,пермь,попасть,приезжать,приём,трактор,транспорт
0,Лысьвенский городской округ,Благоустройство,добрый день сегодня мать идти улица ленин дом ...,★ Ямы во дворах,0.577350,0.577350,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Министерство социального развития ПК,Социальное обслуживание и защита,пермь пермь ноябрь год работать социальный так...,Оказание гос. соц. помощи,0.000000,0.000000,0.000000,0.000000,0.894427,0.000000,0.000000,0.000000,0.000000,0.447214
2,Министерство социального развития ПК,Социальное обслуживание и защита,добрый день сказать пожалуйста подовал пособие...,Дети и многодетные семьи,0.894427,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Город Пермь,Общественный транспорт,каждый человек остановка хотеть укрыться непог...,Содержание остановок,0.000000,0.000000,0.000000,0.400773,0.000000,0.000000,0.000000,0.000000,0.000000,0.916177
4,Министерство здравоохранения,Здравоохранение/Медицина,березники сын привить откоронавирус заболеть п...,Технические проблемы с записью на прием к врачу,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.000000,0.000000
5,АО ПРО ТКО,Мусор/Свалки/ТКО,добрый день подсказать тариф повыситься обраще...,Плата за вывоз ТКО,0.707107,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Министерство образования,Образование,здравствуйте уточнить какой погода ребёнок нач...,Образовательный процесс,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Министерство здравоохранения,Здравоохранение/Медицина,пермский диспансеризация,Диспансеризация,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Министерство здравоохранения,Здравоохранение/Медицина,здравствуйте сказать пожалуйста сколько время ...,★ Просьбы о лечении,0.000000,0.000000,0.962119,0.000000,0.272630,0.000000,0.000000,0.000000,0.000000,0.000000
9,ИГЖН ПК,Дороги,хотеть узнать чайковский управляйка телефон по...,★ Нарушение правил очистки дорог от снега и на...,0.000000,0.000000,0.000000,0.330853,0.000000,0.000000,0.667284,0.000000,0.667284,0.000000
